In [ ]:
import tempfile

import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.externals import joblib

tf.logging.set_verbosity(tf.logging.INFO)


In [ ]:
#Load data and reduce the number of features for the baseline models

train = pd.read_csv('../../wl2522/project/dataset/train.csv', dtype={'source_system_tab': str})
test = pd.read_csv('../../wl2522/project/dataset/test.csv', dtype={'source_system_tab': str })
members = pd.read_csv('../../wl2522/project/dataset/members.csv', dtype={'msno': str, 'city': str, 'registered_via': str})
songs = pd.read_csv('../../wl2522/project/dataset/songs.csv', dtype={'genre_ids': str, 'language': str, 'song_length': int})

#Infer a missing value based on other features

songs.loc[605127, 'language'] = '31.0'

#Impute missing values

train.fillna(value='unknown', axis=1, inplace=True)
test.fillna(value='unknown', axis=1, inplace=True)
members.fillna(value='unknown', axis=1, inplace=True)
songs.fillna(value='unknown', axis=1, inplace=True)

#Convert feature columns related to member registration to datetime format

members['expiration_date'] = pd.to_datetime(members['expiration_date'], format='%Y%m%d')
members['registration_init_time'] = pd.to_datetime(members['registration_init_time'], format='%Y%m%d')

#Create a feature indicating the number of days a member was registered

members['reg_duration'] = (members['expiration_date'] - members['registration_init_time']).dt.days


In [ ]:
#Merge the training and test data with song and member data

train_set = train.merge(songs, on='song_id')
train_set = train_set.merge(members, on='msno')
test_set = test.merge(songs, on='song_id', how='left')
test_set = test_set.merge(members, on='msno', how='left')

#Separate the submission ids from the test set

ids = test_set['id']
test_set.drop('id', axis=1, inplace=True)

#Impute missing values for specific features in the merged training and test sets with an integer value 

msno_vocabs = train_set['msno'].unique()
msno_map = { msno_vocabs[i]:i+1 for i in range(len(msno_vocabs))}
train_set['msno'] = train_set['msno'].map(msno_map).fillna(-99).astype(int)
test_set['msno'] = test_set['msno'].map(msno_map).fillna(-99).astype(int)

song_vocabs = train_set['song_id'].unique()
song_map = { song_vocabs[i]:i+1 for i in range(len(song_vocabs))}
train_set['song_id'] = train_set['song_id'].map(song_map).fillna(-99).astype(int)
test_set['song_id'] = test_set['song_id'].map(song_map).fillna(-99).astype(int)

tab_vocabs = train_set['source_system_tab'].unique()
tab_map = { tab_vocabs[i]:i+1 for i in range(len(tab_vocabs))}
train_set['source_system_tab'] = train_set['source_system_tab'].map(tab_map).fillna(-99).astype(int)
test_set['source_system_tab'] = test_set['source_system_tab'].map(tab_map).fillna(-99).astype(int)

screen_vocabs = train_set['source_screen_name'].unique()
screen_map = { screen_vocabs[i]:i+1 for i in range(len(screen_vocabs))}
train_set['source_screen_name'] = train_set['source_screen_name'].map(screen_map).fillna(-99).astype(int)
test_set['source_screen_name'] = test_set['source_screen_name'].map(screen_map).fillna(-99).astype(int)

sctype_vocabs = train_set['source_type'].unique()
sctype_map = { sctype_vocabs[i]:i+1 for i in range(len(sctype_vocabs))}
train_set['source_type'] = train_set['source_type'].map(sctype_map).fillna(-99).astype(int)
test_set['source_type'] = test_set['source_type'].map(sctype_map).fillna(-99).astype(int)

gid_vocabs = train_set['genre_ids'].unique()
gid_map = { gid_vocabs[i]:i+1 for i in range(len(gid_vocabs))}
train_set['genre_ids'] = train_set['genre_ids'].map(gid_map).fillna(-99).astype(int)
test_set['genre_ids'] = test_set['genre_ids'].map(gid_map).fillna(-99).astype(int)

gender_vocabs = train_set['gender'].unique()
gender_map = { gender_vocabs[i]:i+1 for i in range(len(gender_vocabs))}
train_set['gender'] = train_set['gender'].map(gender_map).fillna(-99).astype(int)
test_set['gender'] = test_set['gender'].map(gender_map).fillna(-99).astype(int)

city_vocabs = train_set['city'].unique()
city_map = { city_vocabs[i]:i+1 for i in range(len(city_vocabs))}
train_set['city'] = train_set['city'].map(city_map).fillna(-99).astype(int)
test_set['city'] = test_set['city'].map(city_map).fillna(-99).astype(int)

lan_vocabs = train_set['language'].unique()
lan_map = { lan_vocabs[i]:i+1 for i in range(len(lan_vocabs))}
train_set['language'] = train_set['language'].map(lan_map).fillna(-99).astype(int)
test_set['language'] = test_set['language'].map(lan_map).fillna(-99).astype(int)

#Impute missing values for the remaining features with a string

train_set.fillna(value='unknown', axis=1, inplace=True)
test_set.fillna(value='unknown', axis=1, inplace=True)

#Impute missing song lengths with an integer value to avoid errors due to conflicting data types

test_set['song_length'] = test_set['song_length'].replace('unknown', -99)


In [ ]:
#load the pretrained GBDT model in order to use its predictions as a feature in the final model

train_wide = pd.read_csv('tmp/train.csv',index_col = 0)
test_wide = pd.read_csv('tmp/test.csv',index_col = 0)
train_wide.drop('target', axis=1, inplace=True)

feature_trans = joblib.load('./tmp/lgb.pkl')
tr_features = feature_trans.predict(train_wide.values,pred_leaf=True)
tes_features = feature_trans.predict(test_wide.values,pred_leaf=True)

width = tr_features.shape[1]
trans_feature = ['trans_feature' + str(i) for i in range(width)]
train_trans = pd.DataFrame()

test_trans = pd.DataFrame()

for i in range(len(trans_feature)):
    train_trans[trans_feature[i]] = tr_features[:,i]
    test_trans[trans_feature[i]] = tes_features[:,i]
    

In [ ]:
#Concatenate the original dataset with the GBDT model predictions

train_dw = pd.concat([train_set,train_trans],axis=1)
test_dw = pd.concat([test_set,test_trans],axis=1)

#Impute values for missing song data

train_dw['song_length'] = pd.to_numeric(train_dw['song_length'],
                                        downcast='integer', errors='coerce').fillna(0)
test_dw['song_length'] = pd.to_numeric(test_dw['song_length'],
                                       downcast='integer', errors='coerce').fillna(0)
test_dw['song_length'].astype('int32')
train_dw['song_length'].astype('int32')

train_dw.to_csv('tmp/train_dw.csv')
test_dw.to_csv('tmp/test_dw.csv')


In [2]:
train_dw = pd.read_csv('../../KKBox/tmp/train_dw.csv')
test_dw = pd.read_csv('../../KKBox/tmp/test_dw.csv')

#Convert the song length values to integer to avoid data type errors

test_dw['song_length'] = test_dw['song_length'].astype('int32')
train_dw['song_length'] = train_dw['song_length'].astype('int32')


In [3]:
#Shuffle the data and split off 20% of the training set for use as a validation set

split_ratio = 0.8

train_dw = train_dw.sample(frac=1, random_state=6)
val_dw = train_dw[int(split_ratio*train_dw.shape[0]):]
train_dw = train_dw[:int(split_ratio*train_dw.shape[0])]

#Separate the labels from the training and validation sets

y_train = train_dw['target']
train_dw.drop('target', axis=1, inplace=True)

y_val = val_dw['target']
val_dw.drop('target', axis=1, inplace=True)

train_set = train_dw
test_set = test_dw


In [4]:
#Designate the target feature name and the features to be used in the dataset

FEATURES = ['msno', 'gender', 'city', 'bd',
            'song_id', 'language', 'genre_ids', 'composer', 'lyricist',
            'source_system_tab', 'source_screen_name', 'source_type', 'reg_duration']

#Include the GBDT predictions as features in the final model (one for each tree in the GBDT model, 32 leaves per tree)

width = 60
trans_feature = ['trans_feature' + str(i) for i in range(width)]
FEATURES.extend(trans_feature)

#Use the feature_column module to input each feature column into the model

target = tf.feature_column.categorical_column_with_identity(key='target', num_buckets=2)

length = tf.feature_column.numeric_column(key='song_length',
                                          default_value=-1,
                                          dtype=tf.int32)

duration = tf.feature_column.numeric_column(key='reg_duration',
                                            default_value=-1,
                                            dtype=tf.int32)

gender = tf.feature_column.categorical_column_with_vocabulary_list(key='gender',
                                                                   vocabulary_list=train_dw['gender'].unique(),
                                                                   dtype=tf.int32,
                                                                   default_value=-99)

city = tf.feature_column.categorical_column_with_vocabulary_list(key='city',
                                                          vocabulary_list=train_dw['city'].unique(),
                                                          dtype=tf.int32,
                                                          default_value=-99)

language = tf.feature_column.categorical_column_with_vocabulary_list(key='language',
                                                                     vocabulary_list=train_dw['language'].unique(),
                                                                     dtype=tf.int32,
                                                                     default_value=-99)

composer = tf.feature_column.categorical_column_with_vocabulary_list(key='composer',
                                                                     vocabulary_list=songs['composer'].unique(),
                                                                     dtype=tf.string,
                                                                     default_value=-99)

lyricist = tf.feature_column.categorical_column_with_vocabulary_list(key='lyricist',
                                                                     vocabulary_list=songs['lyricist'].unique(),
                                                                     dtype=tf.string,
                                                                     default_value=-99)

tab = tf.feature_column.categorical_column_with_vocabulary_list(key='source_system_tab',
                                                                vocabulary_list=train_set['source_system_tab'].unique(),
                                                                dtype=tf.int32,
                                                                default_value=-99)

screen = tf.feature_column.categorical_column_with_vocabulary_list(key='source_screen_name',
                                                                   vocabulary_list=train_set['source_screen_name'].unique(),
                                                                   dtype=tf.int32,
                                                                   default_value=-99)

source = tf.feature_column.categorical_column_with_vocabulary_list(key='source_type',
                                                                   vocabulary_list=train_set['source_type'].unique(),
                                                                   dtype=tf.int32,
                                                                   default_value=-99)

#Bucket categorical features with many unique categories using a hash table with a size of approximately (n/0.8)*2

msno = tf.feature_column.categorical_column_with_hash_bucket(key='msno',
                                                               hash_bucket_size=90000,
                                                               dtype=tf.int32)

song_id = tf.feature_column.categorical_column_with_hash_bucket(key='song_id',
                                                             hash_bucket_size=6000000,
                                                             dtype=tf.int32)

hashed_genre = tf.feature_column.categorical_column_with_hash_bucket(key='genre_ids',
                                                                     hash_bucket_size=3000,
                                                                     dtype=tf.int32)

#Perform one hot encoding on categorical features with few unique values

indicator_gender = tf.feature_column.indicator_column(gender)
indicator_city = tf.feature_column.indicator_column(city)
indicator_language = tf.feature_column.indicator_column(language)
indicator_tab = tf.feature_column.indicator_column(tab)
indicator_screen = tf.feature_column.indicator_column(screen)
indicator_source = tf.feature_column.indicator_column(source)

#Embed the categorical feature with <100 unique categories into dense vectors with approximately log2(n) dimensions

embedded_genre = tf.feature_column.embedding_column(hashed_genre, dimension=10)
embedded_song = tf.feature_column.embedding_column(song_id, dimension=22)
embedded_msno = tf.feature_column.embedding_column(msno, dimension=15)
embedded_composer = tf.feature_column.embedding_column(composer, dimension=18)
embedded_lyricist = tf.feature_column.embedding_column(lyricist, dimension=17)

#Bucket member age into age ranges, with nonsensical values going into the 0-14 or the >80 buckets

age = tf.feature_column.numeric_column(key='bd',
                                       default_value=0,
                                       dtype=tf.int32)

age_bucket = tf.feature_column.bucketized_column(age, boundaries=[0, 14, 20, 30, 40, 50, 80])

#Assign features to be used in either the wide or the deep model

wide_columns =  [tf.feature_column.categorical_column_with_vocabulary_list(key='trans_feature' + str(i),
                                                                   vocabulary_list=list(range(2**5)),
                                                                   dtype=tf.int32,
                                                                   default_value=-99) for i in range(60)]

deep_columns = [indicator_gender, indicator_city, indicator_language,
                indicator_tab, indicator_screen, indicator_source,
                embedded_genre, embedded_msno, embedded_song,
                embedded_composer, embedded_lyricist,
                duration, age_bucket]


In [18]:
#Create an instance of the TensorFlow estimator for the specified model

def build_estimator(model_dir, model_type):
    if model_type == 'wide':
        model = tf.estimator.LinearClassifier(model_dir=model_dir,
                                              feature_columns=wide_columns + cross_columns)

    elif model_type == 'deep':
        model = tf.estimator.DNNClassifier(model_dir=model_dir,
                                           feature_columns=deep_columns,
                                           hidden_units=[1024, 512, 256],
                                           optimizer=tf.train.AdamOptimizer(learning_rate=0.001,
                                                                            name='Adam'),
                                          config = tf.estimator.RunConfig(save_checkpoints_steps = 3000))

    elif model_type == 'combined':
        model = tf.estimator.DNNLinearCombinedClassifier(model_dir=model_dir,
                                                         linear_feature_columns=wide_columns + cross_columns,
                                                         dnn_feature_columns=deep_columns,
                                                         dnn_hidden_units=[1024, 512, 256],
                                                        config = tf.estimator.RunConfig(save_checkpoints_steps = 3000))

    return model


In [19]:
#Create an input function that takes the dataset and serves it to the model in minibatches

def input_fn(X, y, mode, batch_size):
    X.fillna(value='unknown', axis=1, inplace=True)    

    if mode == 'train':
        return tf.estimator.inputs.numpy_input_fn(x={k: X[k].values for k in FEATURES},
                                                    y=y.values,
                                                    batch_size=batch_size,
                                                    num_epochs=None,
                                                    shuffle=True,
                                                    num_threads=8)

    elif mode == 'eval':
        return tf.estimator.inputs.numpy_input_fn(x={k: X[k].values for k in FEATURES},
                                                    y=y.values,
                                                    batch_size=batch_size,
                                                    num_epochs=None,
                                                    shuffle=False,
                                                    num_threads=1)
    
    elif mode == 'predict':
        return tf.estimator.inputs.numpy_input_fn(x={k: X[k].values for k in FEATURES},
                                                    batch_size=batch_size,
                                                    num_epochs=1,
                                                    shuffle=False,
                                                    num_threads=1)
    

In [20]:
def train_model(model_dir, model_type, train_steps, X_train, y_train, X_test, y_test, batch_size):

#Create a temporary directory to store the model if no model directory argument is given

    model_dir = tempfile.mkdtemp() if not model_dir else model_dir

    print('build_estimator')
    model = build_estimator(model_dir, model_type)

    print('train start')

#Wrap the estimator in an experiment so that metrics are calculated on the training set during training

    experiment = tf.contrib.learn.Experiment(estimator=model,
                                             train_input_fn=input_fn(X_train,
                                                                     y_train,
                                                                     mode='train',
                                                                     batch_size=batch_size),
                                            eval_input_fn=input_fn(X_train,
                                                                   y_train,
                                                                   mode='eval',
                                                                   batch_size=batch_size),
                                            train_steps=train_steps,
                                            min_eval_frequency=1000)
    
    experiment.train_and_evaluate()
    
#Evaluate the trained model on a separate validation set in n/batch_size steps

    model.evaluate(input_fn=input_fn(X_test, y_test, mode='eval', batch_size=batch_size))

    print('end!')

    return model


In [22]:
deep_model = train_model(model_dir='./test-dw-model/', model_type='combined', train_steps=250000,
                         X_train=train_dw, y_train=y_train,
                         X_test=val_dw, y_test=y_val,
                         batch_size=100)


build_estimator
INFO:tensorflow:Using config: {'_model_dir': './test-dw-model/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 3000, '_save_checkpoints_secs': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f36fa05b518>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
train start
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ./test-dw-model/model.ckpt.
INFO:tensorflow:loss = 6732.49, step = 1
INFO:tensorflow:global_step/sec: 8.30459
INFO:tensorflow:loss = 294.486, step = 101 (12.043 sec)
INFO:tensorflow:global_step/sec: 9.96365
INFO:tensorflow:loss = 380.854, step = 201 (10.037 sec)
INFO:tensorflow:global_step/sec: 9.96152
INFO:tensorflow:loss = 382.724, step

INFO:tensorflow:global_step/sec: 9.99927
INFO:tensorflow:loss = 64.7157, step = 3201 (10.001 sec)
INFO:tensorflow:global_step/sec: 10.0164
INFO:tensorflow:loss = 71.9676, step = 3301 (9.984 sec)
INFO:tensorflow:global_step/sec: 10.066
INFO:tensorflow:loss = 75.938, step = 3401 (9.934 sec)
INFO:tensorflow:global_step/sec: 10.0036
INFO:tensorflow:loss = 63.6763, step = 3501 (9.997 sec)
INFO:tensorflow:global_step/sec: 10.0115
INFO:tensorflow:loss = 71.3921, step = 3601 (9.989 sec)
INFO:tensorflow:global_step/sec: 9.96462
INFO:tensorflow:loss = 69.8669, step = 3701 (10.035 sec)
INFO:tensorflow:global_step/sec: 9.9084
INFO:tensorflow:loss = 68.4996, step = 3801 (10.093 sec)
INFO:tensorflow:global_step/sec: 9.99162
INFO:tensorflow:loss = 77.3612, step = 3901 (10.007 sec)
INFO:tensorflow:Starting evaluation at 2017-12-04-00:41:59
INFO:tensorflow:Restoring parameters from ./test-dw-model/model.ckpt-3001
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Eval

INFO:tensorflow:Starting evaluation at 2017-12-04-00:47:19
INFO:tensorflow:Restoring parameters from ./test-dw-model/model.ckpt-6001
INFO:tensorflow:Evaluation [1/100]
INFO:tensorflow:Evaluation [2/100]
INFO:tensorflow:Evaluation [3/100]
INFO:tensorflow:Evaluation [4/100]
INFO:tensorflow:Evaluation [5/100]
INFO:tensorflow:Evaluation [6/100]
INFO:tensorflow:Evaluation [7/100]
INFO:tensorflow:Evaluation [8/100]
INFO:tensorflow:Evaluation [9/100]
INFO:tensorflow:Evaluation [10/100]
INFO:tensorflow:Evaluation [11/100]
INFO:tensorflow:Evaluation [12/100]
INFO:tensorflow:Evaluation [13/100]
INFO:tensorflow:Evaluation [14/100]
INFO:tensorflow:Evaluation [15/100]
INFO:tensorflow:Evaluation [16/100]
INFO:tensorflow:Evaluation [17/100]
INFO:tensorflow:Evaluation [18/100]
INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorf

INFO:tensorflow:Evaluation [19/100]
INFO:tensorflow:Evaluation [20/100]
INFO:tensorflow:Evaluation [21/100]
INFO:tensorflow:Evaluation [22/100]
INFO:tensorflow:Evaluation [23/100]
INFO:tensorflow:Evaluation [24/100]
INFO:tensorflow:Evaluation [25/100]
INFO:tensorflow:Evaluation [26/100]
INFO:tensorflow:Evaluation [27/100]
INFO:tensorflow:Evaluation [28/100]
INFO:tensorflow:Evaluation [29/100]
INFO:tensorflow:Evaluation [30/100]
INFO:tensorflow:Evaluation [31/100]
INFO:tensorflow:Evaluation [32/100]
INFO:tensorflow:Evaluation [33/100]
INFO:tensorflow:Evaluation [34/100]
INFO:tensorflow:Evaluation [35/100]
INFO:tensorflow:Evaluation [36/100]
INFO:tensorflow:Evaluation [37/100]
INFO:tensorflow:Evaluation [38/100]
INFO:tensorflow:Evaluation [39/100]
INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [40/100]
INFO:tensorflow:Evaluation [41/100]
INFO:tensorflow:Evaluation [42/100]
INFO:tensorflow:Evaluation [43/100]
INFO:tensorflow:Evaluation [44/100]
INFO:tensorflow:Evaluation [45/100]
INFO:tensorflow:Evaluation [46/100]
INFO:tensorflow:Evaluation [47/100]
INFO:tensorflow:Evaluation [48/100]
INFO:tensorflow:Evaluation [49/100]
INFO:tensorflow:Evaluation [50/100]
INFO:tensorflow:Evaluation [51/100]
INFO:tensorflow:Evaluation [52/100]
INFO:tensorflow:Evaluation [53/100]
INFO:tensorflow:Evaluation [54/100]
INFO:tensorflow:Evaluation [55/100]
INFO:tensorflow:Evaluation [56/100]
INFO:tensorflow:Evaluation [57/100]
INFO:tensorflow:Evaluation [58/100]
INFO:tensorflow:Evaluation [59/100]
INFO:tensorflow:Evaluation [60/100]
INFO:tensorflow:Evaluation [61/100]
INFO:tensorflow:Evaluation [62/100]
INFO:tensorflow:Evaluation [63/100]
INFO:tensorflow:Evaluation [64/100]
INFO:tensorflow:Evaluation [65/100]
INFO:tensorflow:Evaluation [66/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [62/100]
INFO:tensorflow:Evaluation [63/100]
INFO:tensorflow:Evaluation [64/100]
INFO:tensorflow:Evaluation [65/100]
INFO:tensorflow:Evaluation [66/100]
INFO:tensorflow:Evaluation [67/100]
INFO:tensorflow:Evaluation [68/100]
INFO:tensorflow:Evaluation [69/100]
INFO:tensorflow:Evaluation [70/100]
INFO:tensorflow:Evaluation [71/100]
INFO:tensorflow:Evaluation [72/100]
INFO:tensorflow:Evaluation [73/100]
INFO:tensorflow:Evaluation [74/100]
INFO:tensorflow:Evaluation [75/100]
INFO:tensorflow:Evaluation [76/100]
INFO:tensorflow:Evaluation [77/100]
INFO:tensorflow:Evaluation [78/100]
INFO:tensorflow:Evaluation [79/100]
INFO:tensorflow:Evaluation [80/100]
INFO:tensorflow:Evaluation [81/100]
INFO:tensorflow:Evaluation [82/100]
INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [

INFO:tensorflow:Evaluation [83/100]
INFO:tensorflow:Evaluation [84/100]
INFO:tensorflow:Evaluation [85/100]
INFO:tensorflow:Evaluation [86/100]
INFO:tensorflow:Evaluation [87/100]
INFO:tensorflow:Evaluation [88/100]
INFO:tensorflow:Evaluation [89/100]
INFO:tensorflow:Evaluation [90/100]
INFO:tensorflow:Evaluation [91/100]
INFO:tensorflow:Evaluation [92/100]
INFO:tensorflow:Evaluation [93/100]
INFO:tensorflow:Evaluation [94/100]
INFO:tensorflow:Evaluation [95/100]
INFO:tensorflow:Evaluation [96/100]
INFO:tensorflow:Evaluation [97/100]
INFO:tensorflow:Evaluation [98/100]
INFO:tensorflow:Evaluation [99/100]
INFO:tensorflow:Evaluation [100/100]
INFO:tensorflow:Finished evaluation at 2017-12-04-01:08:46
INFO:tensorflow:Saving dict for global step 18001: accuracy = 0.6455, accuracy_baseline = 0.5106, auc = 0.699485, auc_precision_recall = 0.691022, average_loss = 0.630664, global_step = 18001, label/mean = 0.5106, loss = 63.0664, prediction/mean = 0.479673
INFO:tensorflow:Validation (step 19

INFO:tensorflow:Validation (step 22000): accuracy = 0.6478, accuracy_baseline = 0.5106, auc = 0.700462, auc_precision_recall = 0.691738, average_loss = 0.62779, label/mean = 0.5106, loss = 62.779, prediction/mean = 0.512005, global_step = 21001
INFO:tensorflow:global_step/sec: 3.77945
INFO:tensorflow:loss = 61.5925, step = 22001 (26.459 sec)
INFO:tensorflow:global_step/sec: 9.95173
INFO:tensorflow:loss = 67.9246, step = 22101 (10.049 sec)
INFO:tensorflow:global_step/sec: 9.88653
INFO:tensorflow:loss = 62.6627, step = 22201 (10.114 sec)
INFO:tensorflow:global_step/sec: 10.0367
INFO:tensorflow:loss = 63.4772, step = 22301 (9.964 sec)
INFO:tensorflow:global_step/sec: 9.98474
INFO:tensorflow:loss = 57.3496, step = 22401 (10.015 sec)
INFO:tensorflow:global_step/sec: 9.95476
INFO:tensorflow:loss = 64.7447, step = 22501 (10.045 sec)
INFO:tensorflow:global_step/sec: 10.0988
INFO:tensorflow:loss = 65.8755, step = 22601 (9.902 sec)
INFO:tensorflow:global_step/sec: 10.0029
INFO:tensorflow:loss = 

INFO:tensorflow:loss = 65.7728, step = 25501 (9.992 sec)
INFO:tensorflow:global_step/sec: 10.0332
INFO:tensorflow:loss = 67.213, step = 25601 (9.966 sec)
INFO:tensorflow:global_step/sec: 10.0257
INFO:tensorflow:loss = 65.0577, step = 25701 (9.974 sec)
INFO:tensorflow:global_step/sec: 10.0916
INFO:tensorflow:loss = 69.4205, step = 25801 (9.909 sec)
INFO:tensorflow:global_step/sec: 10.0252
INFO:tensorflow:loss = 74.1869, step = 25901 (9.975 sec)
INFO:tensorflow:global_step/sec: 10.0304
INFO:tensorflow:loss = 61.4962, step = 26001 (9.970 sec)
INFO:tensorflow:global_step/sec: 10.1122
INFO:tensorflow:loss = 62.5022, step = 26101 (9.889 sec)
INFO:tensorflow:global_step/sec: 10.1306
INFO:tensorflow:loss = 61.6238, step = 26201 (9.871 sec)
INFO:tensorflow:global_step/sec: 10.1276
INFO:tensorflow:loss = 69.1099, step = 26301 (9.874 sec)
INFO:tensorflow:global_step/sec: 10.1144
INFO:tensorflow:loss = 67.3911, step = 26401 (9.887 sec)
INFO:tensorflow:global_step/sec: 10.1049
INFO:tensorflow:loss 

KeyboardInterrupt: 

In [ ]:
#Make predictions on the test set and write them to a csv file

predictions = deep_model.predict(input_fn=input_fn(test_set, None, mode='predict',
                                                   batch_size=10000))

submission = list()

for row in predictions:
    submission.append(row['probabilities'][1])
    
ids = pd.read_csv("../../KKBox/input/test.csv")['id'].values
result_df = pd.DataFrame()

result_df['id'] = ids
result_df['target'] = np.array(submission)
result_df.to_csv('dw.csv.gz', compression = 'gzip',
                 index=False, float_format = '%.5f')
